# Library

In [ ]:
import cv2 as cv
import numpy as np
import math

# Moment de HU

In [4]:
# le moment d'ordre p ,q
def momemtpq(ib, p, q):
    s = 0
    h,l = ib.shape
    for x in range(h):
        for y in range(l):
            s = s + (x**p)*(y**q)*ib[x, y]
    return s

# moments centraux
def mupq(ib, p, q):
    mu = 0
    l,h = ib.shape
    x_ = momemtpq(ib, 1, 0)/momemtpq(ib, 0, 0)
    y_ = momemtpq(ib, 0, 1)/momemtpq(ib, 0, 0)
    for x in range(l):
        for y in range(h):
            mu = mu + (((x-x_)**p)*((y-y_)**q))*ib[x, y]
    return mu


# Les moments centraux normalisés
def alpha_pq(ib, p, q):
    lambda_ = ((p+q+1)/2) + 1
    alpha = mupq(ib, p, q)/(mupq(ib, 0, 0)**lambda_)
    return alpha;


def momentHU(ib,class_num):
    phi = np.zeros([8], dtype=float)
    
    a_0_2 = alpha_pq(ib, 0, 2)
    a_0_3 = alpha_pq(ib, 0, 3)
    a_1_1 = alpha_pq(ib, 1, 1)
    a_1_2 = alpha_pq(ib, 1, 2)
    a_2_0 = alpha_pq(ib, 2, 0)
    a_2_1 = alpha_pq(ib, 2, 1)
    a_3_0 = alpha_pq(ib, 3, 0)
    
    phi[0] = a_2_0 - a_0_2
    phi[1] = (a_2_0 - a_0_2)**2 + (4*a_1_1)**2
    phi[2] = (a_3_0 - a_1_2)**2 +(3*a_1_2 - a_0_3)**2
    phi[3] = (a_3_0 + a_1_2)**2 + (a_2_1+a_0_3)**2
    
    phi[4] = (a_3_0 - 3*a_1_2)*(a_3_0+a_1_2)*((a_3_0+a_1_2)**2 - 3*(a_2_1+a_0_3)**2) +(3*a_2_1-a_0_3)*(a_2_1+a_0_3)*(3*(a_3_0+a_1_2)**2-((a_2_1+a_0_3)**2))
   
    phi[5] = (a_2_0-a_0_2)*((a_3_0+a_1_2)**2-(a_2_1+a_0_3)**2)+4*a_1_1*(a_3_0+a_1_2)*(a_2_1+a_0_3)
   
    phi[6] = (3*a_2_1-a_3_0)*(a_3_0+a_1_2)*((a_3_0+a_1_2)**2-3*((a_2_1+a_0_3)**2))+(3*a_1_2-a_0_3)*(a_2_1+a_0_3)*((3*(a_3_0+a_1_2)**2)-((a_2_1+a_0_3)**2))
    phi[7] = class_num
    return phi;


In [3]:
def momentHU_rn(ib):
    phi = np.zeros([7], dtype=float)
    
    a_0_2 = alpha_pq(ib, 0, 2)
    a_0_3 = alpha_pq(ib, 0, 3)
    a_1_1 = alpha_pq(ib, 1, 1)
    a_1_2 = alpha_pq(ib, 1, 2)
    a_2_0 = alpha_pq(ib, 2, 0)
    a_2_1 = alpha_pq(ib, 2, 1)
    a_3_0 = alpha_pq(ib, 3, 0)
    
    phi[0] = a_2_0 - a_0_2
    phi[1] = (a_2_0 - a_0_2)**2 + (4*a_1_1)**2
    phi[2] = (a_3_0 - a_1_2)**2 +(3*a_1_2 - a_0_3)**2
    phi[3] = (a_3_0 + a_1_2)**2 + (a_2_1+a_0_3)**2
    
    phi[4] = (a_3_0 - 3*a_1_2)*(a_3_0+a_1_2)*((a_3_0+a_1_2)**2 - 3*(a_2_1+a_0_3)**2) +(3*a_2_1-a_0_3)*(a_2_1+a_0_3)*(3*(a_3_0+a_1_2)**2-((a_2_1+a_0_3)**2))
   
    phi[5] = (a_2_0-a_0_2)*((a_3_0+a_1_2)**2-(a_2_1+a_0_3)**2)+4*a_1_1*(a_3_0+a_1_2)*(a_2_1+a_0_3)
   
    phi[6] = (3*a_2_1-a_3_0)*(a_3_0+a_1_2)*((a_3_0+a_1_2)**2-3*((a_2_1+a_0_3)**2))+(3*a_1_2-a_0_3)*(a_2_1+a_0_3)*((3*(a_3_0+a_1_2)**2)-((a_2_1+a_0_3)**2))
    return phi;

# Moment de tchibichef

In [ ]:
# 4 squzred-

def squared_norm(n,N):
    stop1 = n+N
    stop2 = N -n -1
    sum1 = 1
    for i in range(1,stop1+1):
        sum1 = sum1*i
    sum2 = 1
    for i in range(1,stop2+1):
        sum2 = sum2*i
    sum3 = 2 * n + 1
    pnN = sum1/(sum3*sum2)
    return pnN

def pochhammer(a,k):
    a = 1
    for i in range(1,k+1):
        a = a*(a + (i-1))
    return a


def polynomial (n,M,x):
    
    t1 = pochhammer(1-M,n)
    t2 = math.sqrt( squared_norm(n,M) )
    sum = 0
    for i in range(0,n+1):
        t3 = pochhammer(-n,i)
        t4 = pochhammer(-x,i)
        t5 = pochhammer(1+n,i)
        t6 = math.factorial(i)**2
        t7 = pochhammer(1-M,i)
        sum =sum+((t3*t4*t5)/(t6*t7))
    sum = (t1/t2)*sum        
    
    return sum

def momentNM(img, n,m):
    sum = 0
    M,N = img.shape
    for i in  range(0,M):
        for j in  range(0,N):
            t1 = polynomial(i,n,M)
            t2 = polynomial(j,n,M)
            sum = sum +  (t1*t2*img[i,j])
    return sum
    
def tchebichef(img, ordre):
    T = []
    for i in range(0, ordre+1):
        for j in range(0, ordre+1):
            if (i+j) <=ordre:
                T.append(momentNM(img, i, j))
    return T
